## Schemas

<https://www.kaggle.com/c/tmdb-box-office-prediction/data>

In [65]:
import pandas as pd

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Pyspark course") \
    .getOrCreate()

## Read csv data

In [66]:
films_sdf = spark\
    .read\
    .csv("data/tmdb-box-office-prediction/train.csv", header=True)

In [67]:
films_sdf.printSchema()

root
 |-- id: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- revenue: string (nullable = true)



In [76]:
print(films_sdf.select("cast").limit(10).toPandas().iloc[1]["cast"])

"[{'cast_id': 1, 'character': 'Mia Thermopolis', 'credit_id': '52fe43fe9251416c7502561f', 'gender': 1, 'id': 1813, 'name': 'Anne Hathaway', 'order': 0, 'profile_path': '/jUMOKwSUBnTcMeN1HfhutiY49Ad.jpg'}, {'cast_id': 2, 'character': 'Queen Clarisse Renaldi', 'credit_id': '52fe43fe9251416c75025623', 'gender': 1, 'id': 5823, 'name': 'Julie Andrews', 'order': 1, 'profile_path': '/6t61jkmfSA6nbYRCKR9s97CgUN6.jpg'}, {'cast_id': 3, 'character': 'Joe', 'credit_id': '52fe43fe9251416c75025627', 'gender': 2, 'id': 1210, 'name': 'H√©ctor Elizondo', 'order': 2, 'profile_path': '/48UNfVFZVr0jyMIlLPhzm8IIM7f.jpg'}, {'cast_id': 4, 'character': 'Viscount Mabrey', 'credit_id': '52fe43fe9251416c7502562b', 'gender': 2, 'id': 655, 'name': 'John Rhys-Davies', 'order': 3, 'profile_path': '/zZ67PuoFfik9QlZyfaEsFBC1yVJ.jpg'}, {'cast_id': 5, 'character': 'Lilly Moscovitz', 'credit_id': '52fe43fe9251416c7502562f', 'gender': 1, 'id': 33656, 'name': 'Heather Matarazzo', 'order': 4, 'profile_path': '/xcwR8aPuSkUCD

In [64]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F

films_sdf = films_sdf\
    .withColumn("id", films_sdf.id.cast(IntegerType()))
    
    
films_sdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- 

In [7]:
films_sdf.select("genres").show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------+
|genres                                                                                                                        |
+------------------------------------------------------------------------------------------------------------------------------+
|[{'id': 35, 'name': 'Comedy'}]                                                                                                |
|[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]|
|[{'id': 18, 'name': 'Drama'}]                                                                                                 |
|[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]                                                                 |
|[{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]                                  

In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType

schema = ArrayType(
    StructType([
        StructField("id", IntegerType()),
        StructField("name", StringType())
    ])
)

films_sdf = films_sdf\
    .withColumn("genres", F.from_json(F.col("genres"), schema))

In [9]:
films_sdf.select("genres").show(5, truncate=False)

+--------------------------------------------------------------+
|genres                                                        |
+--------------------------------------------------------------+
|[[35, Comedy]]                                                |
|[[35, Comedy], [18, Drama], [10751, Family], [10749, Romance]]|
|[[18, Drama]]                                                 |
|[[53, Thriller], [18, Drama]]                                 |
|[[28, Action], [53, Thriller]]                                |
+--------------------------------------------------------------+
only showing top 5 rows



In [10]:
films_sdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- 

In [11]:
films_sdf.select("genres.name").show(5, truncate=False)

+--------------------------------+
|name                            |
+--------------------------------+
|[Comedy]                        |
|[Comedy, Drama, Family, Romance]|
|[Drama]                         |
|[Thriller, Drama]               |
|[Action, Thriller]              |
+--------------------------------+
only showing top 5 rows



In [12]:
films_sdf.select("genres.name").printSchema()

root
 |-- name: array (nullable = true)
 |    |-- element: string (containsNull = true)



## String manipulation

In [13]:
films_sdf2 = spark.read.csv("data/tmdb-box-office-prediction/train.csv", header=True)

In [15]:
films_sdf2.select("genres").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|genres                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{'id': 35, 'name': 'Comedy'}]                                                                                                                                |
|[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]                                |
|[{'id': 18, 'name': 'Drama'}]                                                                                                                                 |
|[{'id': 53, 'name': 'Thriller'}, 

In [20]:
genres_col = films_sdf2\
    .select(
    "id", 
    F.col("genres").substr(F.lit(2), F.length(F.col("genres"))-2).alias("genres"))

genres_col.show(3, truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------+
|id |genres                                                                                                                      |
+---+----------------------------------------------------------------------------------------------------------------------------+
|1  |{'id': 35, 'name': 'Comedy'}                                                                                                |
|2  |{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}|
|3  |{'id': 18, 'name': 'Drama'}                                                                                                 |
+---+----------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [24]:
genres_col\
     .withColumn("genres_array", F.split(F.col("genres"), '(?<=}), '))\
     .head(2)

[Row(id='1', genres="{'id': 35, 'name': 'Comedy'}", genres_array=["{'id': 35, 'name': 'Comedy'}"]),
 Row(id='2', genres="{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}", genres_array=["{'id': 35, 'name': 'Comedy'}", "{'id': 18, 'name': 'Drama'}", "{'id': 10751, 'name': 'Family'}", "{'id': 10749, 'name': 'Romance'}"])]

In [34]:
genre_sdf = genres_col\
    .select("id", F.explode(F.split(F.col("genres"), '(?<=}), ')).alias("genre"))

genre_sdf.show(truncate=False)

+---+---------------------------------+
|id |genre                            |
+---+---------------------------------+
|1  |{'id': 35, 'name': 'Comedy'}     |
|2  |{'id': 35, 'name': 'Comedy'}     |
|2  |{'id': 18, 'name': 'Drama'}      |
|2  |{'id': 10751, 'name': 'Family'}  |
|2  |{'id': 10749, 'name': 'Romance'} |
|3  |{'id': 18, 'name': 'Drama'}      |
|4  |{'id': 53, 'name': 'Thriller'}   |
|4  |{'id': 18, 'name': 'Drama'}      |
|5  |{'id': 28, 'name': 'Action'}     |
|5  |{'id': 53, 'name': 'Thriller'}   |
|6  |{'id': 16, 'name': 'Animation'}  |
|6  |{'id': 12, 'name': 'Adventure'}  |
|6  |{'id': 10751, 'name': 'Family'}  |
|7  |{'id': 27, 'name': 'Horror'}     |
|7  |{'id': 53, 'name': 'Thriller'}   |
|8  |{'id': 99, 'name': 'Documentary'}|
|9  |{'id': 28, 'name': 'Action'}     |
|9  |{'id': 35, 'name': 'Comedy'}     |
|9  |{'id': 10402, 'name': 'Music'}   |
|9  |{'id': 10751, 'name': 'Family'}  |
+---+---------------------------------+
only showing top 20 rows



In [37]:
genre_sdf.select(
    F.col('id'),
    F.regexp_extract(F.col("genre"), "(?<='name':\ ')[A-Z][a-z]+", 0).alias("genre_name")
    ).groupBy("genre_name")\
    .count().orderBy(F.desc("count")).show()

+-----------+-----+
| genre_name|count|
+-----------+-----+
|      Drama| 1527|
|     Comedy| 1018|
|   Thriller|  786|
|     Action|  740|
|    Romance|  569|
|      Crime|  467|
|  Adventure|  435|
|     Horror|  296|
|    Science|  289|
|     Family|  257|
|    Fantasy|  231|
|    Mystery|  224|
|  Animation|  140|
|    History|  132|
|      Music|  100|
|        War|  100|
|Documentary|   87|
|    Western|   43|
|    Foreign|   31|
|           |   18|
+-----------+-----+



## Saving data

In [38]:
films_sdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- 

In [43]:
films_sdf.repartition(2).write.mode('overwrite').parquet("data/tmdb1")

In [44]:
!ls data/tmdb1

part-00000-2a1c97ba-29fd-4c3f-b3d6-fd27cc4b6927-c000.snappy.parquet  _SUCCESS
part-00001-2a1c97ba-29fd-4c3f-b3d6-fd27cc4b6927-c000.snappy.parquet


In [48]:
new_films_sdf = spark.read.parquet("data/tmdb1")

In [50]:
new_films_sdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- 

## Schema

In [51]:
from pyspark.sql.types import StructField, StructType, IntegerType, LongType

schema = \
StructType([
    StructField("id", IntegerType(), False),
    StructField("genres", ArrayType(
        StructType([
            StructField("id", IntegerType()),
            StructField("name", StringType())
        ])))
    ])

In [52]:
new_films_sdf = spark.read.schema(schema=schema).parquet("data/tmdb1")

In [53]:
new_films_sdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)



## Partition

In [54]:
sales_sdf = spark.read.option("header", "true").csv("data/competitive-data-science-predict-future-sales/sales_train.csv.gz")

In [55]:
import pyspark.sql.functions as F
sales_with_iso_dates = sales_sdf\
    .withColumn("date", F.from_unixtime(F.unix_timestamp(F.col("date"), 'dd.MM.yyyy')))\
    .withColumn("year", F.year("date"))\
    .withColumn("month", F.month("date"))\
    .withColumn("day", F.dayofmonth("date"))

In [56]:
sales_with_iso_dates.show()

+-------------------+--------------+-------+-------+----------+------------+----+-----+---+
|               date|date_block_num|shop_id|item_id|item_price|item_cnt_day|year|month|day|
+-------------------+--------------+-------+-------+----------+------------+----+-----+---+
|2013-01-02 00:00:00|             0|     59|  22154|     999.0|         1.0|2013|    1|  2|
|2013-01-03 00:00:00|             0|     25|   2552|     899.0|         1.0|2013|    1|  3|
|2013-01-05 00:00:00|             0|     25|   2552|     899.0|        -1.0|2013|    1|  5|
|2013-01-06 00:00:00|             0|     25|   2554|   1709.05|         1.0|2013|    1|  6|
|2013-01-15 00:00:00|             0|     25|   2555|    1099.0|         1.0|2013|    1| 15|
|2013-01-10 00:00:00|             0|     25|   2564|     349.0|         1.0|2013|    1| 10|
|2013-01-02 00:00:00|             0|     25|   2565|     549.0|         1.0|2013|    1|  2|
|2013-01-04 00:00:00|             0|     25|   2572|     239.0|         1.0|2013

In [57]:
sales_with_iso_dates.write.partitionBy("year", "month", "day").mode('overwrite').parquet("data/tmdb2")

In [58]:
!ls data/tmdb2

 _SUCCESS  'year=2013'	'year=2014'  'year=2015'


In [59]:
!ls data/tmdb2/year=2013/month=3/day=19

part-00000-57d39b5d-b032-4205-9b3f-c1c8c6cf3fdc.c000.snappy.parquet


In [60]:
films_sdf2 = spark.read.parquet("data/tmdb2/year=2013")

In [61]:
films_sdf2.show()

+-------------------+--------------+-------+-------+----------+------------+-----+---+
|               date|date_block_num|shop_id|item_id|item_price|item_cnt_day|month|day|
+-------------------+--------------+-------+-------+----------+------------+-----+---+
|2013-12-28 00:00:00|            11|     25|  17760|    3250.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17792|     349.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17490|     149.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17505|    3799.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17865|     169.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17846|     149.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17918|     149.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25|  17839|     149.0|         1.0|   12| 28|
|2013-12-28 00:00:00|            11|     25

In [62]:
films_sdf3 = spark\
    .read\
    .option("basePath", "data/tmdb2")\
    .parquet(
        "data/tmdb2/year=2014/month=3",
        "data/tmdb2/year=2014/month=4"
    )

In [63]:
films_sdf3.groupBy("year", "month").count().show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2014|    4|77906|
|2014|    3|92733|
+----+-----+-----+

